In [72]:
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk, nltk.stem.porter
import scipy.io as scio
from sklearn import svm

In [4]:
def get_vocab_list():
    vocab_dict = {}
    with open('vocab.txt') as f:
        for line in f:
            (val, key) = line.split()
            vocab_dict[int(val)] = key

    return vocab_dict

In [6]:
email_contents = open('emailSample1.txt', 'r').read()

In [45]:
def process_email(email_contents):

    
    vocab_list = get_vocab_list()

    word_indices = np.array([], dtype=np.int64)

    # ===================== Preprocess Email =====================

    email_contents = email_contents.lower()

    email_contents = re.sub('<[^<>]+>', ' ', email_contents)

    # Any numbers get replaced with the string 'number'
    email_contents = re.sub('[0-9]+', 'number', email_contents)

    # Anything starting with http or https:// replaced with 'httpaddr'
    email_contents = re.sub('(http|https)://[^\s]*', 'httpaddr', email_contents)

    # Strings with "@" in the middle are considered emails --> 'emailaddr'
    email_contents = re.sub('[^\s]+@[^\s]+', 'emailaddr', email_contents)

    # The '$' sign gets replaced with 'dollar'
    email_contents = re.sub('[$]+', 'dollar', email_contents)

    # ===================== Tokenize Email =====================

    stemmer = nltk.stem.porter.PorterStemmer()

    # print('email contents : {}'.format(email_contents))

    tokens = re.split('[@$/#.-:&*+=\[\]?!(){\},\'\">_<;% ]', email_contents)

    for token in tokens:
        token = re.sub('[^a-zA-Z0-9]', '', token)
        token = stemmer.stem(token)

        if len(token) < 1:
            continue

        if token in vocab_list.values():
            index = [key for key, value in vocab_list.items() if value == token]
            word_indices=np.append(word_indices,index)
            
            
    return word_indices           



        

        

    

    

<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\['
<>:18: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\['
C:\Users\Elmostafa\AppData\Local\Temp\ipykernel_27680\223703872.py:18: SyntaxWarning: invalid escape sequence '\s'
  email_contents = re.sub('(http|https)://[^\s]*', 'httpaddr', email_contents)
C:\Users\Elmostafa\AppData\Local\Temp\ipykernel_27680\223703872.py:21: SyntaxWarning: invalid escape sequence '\s'
  email_contents = re.sub('[^\s]+@[^\s]+', 'emailaddr', email_contents)
C:\Users\Elmostafa\AppData\Local\Temp\ipykernel_27680\223703872.py:32: SyntaxWarning: invalid escape sequence '\['
  tokens = re.split('[@$/#.-:&*+=\[\]?!(){\},\'\">_<;% ]', email_contents)


In [64]:
word_indices=process_email(email_contents)

In [65]:
# Total number of words in the dictionary
def email_features(word_indices):
    n = 1899
    features = np.zeros(n + 1)

    features[word_indices]=1
    return features
    




In [66]:
features = email_features(word_indices)

In [74]:
# ===================== Part 3: Train Linear SVM for Spam Classification =====================
# In this section, you will train a linear classifier to determine if an
# email is Spam or Not-spam.
data = scio.loadmat('spamTrain.mat')
X = data['X']
y = data['y'].flatten()
c = 0.1
clf = svm.SVC(C=c, kernel='linear')
clf.fit(X, y)

p = clf.predict(X)

print('Training Accuracy: {}'.format(np.mean(p == y) * 100))

Training Accuracy: 99.825


In [77]:
# ===================== Part 4: Test Spam Classification =====================
# After training the classifier, we can evaluate it on a test set. We have
# included a test set in spamTest.mat

# Load the test dataset
data = scio.loadmat('spamTest.mat')
Xtest = data['Xtest']
ytest = data['ytest'].flatten()

p = clf.predict(Xtest)

print('Test Accuracy: {}'.format(np.mean(p == ytest) * 100))

Test Accuracy: 98.9
